In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# only once
!pip install -q pandas numpy scikit-learn xgboost shap lime matplotlib joblib


In [12]:
"""
credit_risk_pipeline.py
Complete pipeline:
- Load data from credit_risk_sample.csv
- Preprocess (impute, encode, scale)
- Train/tune XGBoost (simple randomized search)
- Report baseline metrics (AUC, F1, precision, recall, confusion)
- Global interpretation: SHAP summary + bar plots
- Local interpretation: LIME explanations for 5 cases (3 high-risk, 2 low-risk) near decision boundary
- Save model, visuals, and a textual report

Assumptions:
- data file: credit_risk_sample.csv (in same directory or set DATA_DIR)
- borderline cases file optional: borderline_cases.csv (if present, picks specified indices)
- target column name default = 'target' (change TARGET_COL if different)
"""

import os
os.listdir()
import warnings
warnings.filterwarnings("ignore")

# ---------- CONFIG ----------
DATA_DIR = "."  # repo root where credit_risk_sample.csv exists
DATA_FILE = os.path.join(DATA_DIR, "/content/drive/MyDrive/credit_risk_sample.csv")
BORDERLINE_FILE = os.path.join(DATA_DIR, "borderline_cases.csv")  # optional
TARGET_COL = "default"  # change if your target column differs
ID_COL = None  # set to 'id' or 'customer_id' if present; else None
RANDOM_STATE = 42
VIS_DIR = "visuals"
MODEL_DIR = "models"
REPORTS_DIR = "reports"
os.makedirs(VIS_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

# ---------- DEPENDENCIES ----------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
import xgboost as xgb
import shap

# LIME import
try:
    from lime.lime_tabular import LimeTabularExplainer
except Exception as e:
    raise RuntimeError("Please pip install lime (pip install lime) before running this script.") from e

# ---------- Helper functions ----------
def safe_read_csv(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"{path} not found. Put the dataset in the project root or set DATA_DIR.")
    return pd.read_csv(path)

def infer_feature_types(df, target_col):
    # separate numeric and categorical automatically
    features = [c for c in df.columns if c != target_col and c != ID_COL]
    num_cols = df[features].select_dtypes(include=['int64','float64']).columns.tolist()
    cat_cols = [c for c in features if c not in num_cols]
    return num_cols, cat_cols

# ---------- Load Data ----------
print("Loading dataset...")
df = safe_read_csv(DATA_FILE)
print("Dataset shape:", df.shape)
if TARGET_COL not in df.columns:
    raise ValueError(f"Target column '{TARGET_COL}' not found in dataset columns: {df.columns.tolist()}")

# If ID column not set, try to detect common ids
if ID_COL is None:
    for candidate in ['id','ID','customer_id','cust_id']:
        if candidate in df.columns:
            ID_COL = candidate
            break

# Quick glance: value counts if target small
print("Target distribution:")
print(df[TARGET_COL].value_counts(dropna=False))

# ---------- Train/Test Split ----------
# Drop rows with missing target
df = df.dropna(subset=[TARGET_COL]).copy()
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL].astype(int)

# If ID col present, preserve it in X for later mapping
if ID_COL and ID_COL in X.columns:
    ids = X[ID_COL].copy()
else:
    ids = pd.Series(np.arange(len(X)), name="index")

# infer feature types
num_cols, cat_cols = infer_feature_types(df, TARGET_COL)
print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)

# Train-test split (stratify)
X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
    X, y, ids, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# ---------- Preprocessing pipeline ----------
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
], remainder='drop')

# Fit preprocessor on train
print("Fitting preprocessor...")
preprocessor.fit(X_train)

# Transform datasets
X_train_p = preprocessor.transform(X_train)
X_test_p = preprocessor.transform(X_test)

# Build feature names after transformation (for SHAP/LIME labeling)
ohe_cols = []
if len(cat_cols) > 0:
    ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
    categories = preprocessor.named_transformers_['cat'].named_steps['onehot'].categories_
    for col, cats in zip(cat_cols, categories):
        for cat in cats:
            ohe_cols.append(f"{col}__{cat}")
feature_names = num_cols + ohe_cols

# ---------- Model training: XGBoost ----------
print("Training XGBoost classifier...")
clf = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False, random_state=RANDOM_STATE)

# quick param grid for RandomizedSearchCV (keeps runtime reasonable)
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

rs = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=10, scoring='roc_auc', cv=3, random_state=RANDOM_STATE, n_jobs=-1, verbose=1)
rs.fit(X_train_p, y_train)
best = rs.best_estimator_
print("Best params:", rs.best_params_)
print("Best CV AUC:", rs.best_score_)

# Predictions & metrics
y_pred_proba = best.predict_proba(X_test_p)[:,1]
y_pred = (y_pred_proba >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_pred_proba)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print("\nTest Metrics:")
print(f"AUC: {auc:.4f}")
print(f"F1: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print("Confusion matrix:\n", cm)
print("\nClassification report:\n", classification_report(y_test, y_pred))

# Save model + preprocessor
joblib.dump(best, os.path.join(MODEL_DIR, "final_model.pkl"))
joblib.dump(preprocessor, os.path.join(MODEL_DIR, "preprocessor.pkl"))
print("Saved model and preprocessor to", MODEL_DIR)

# ---------- Global SHAP analysis ----------
print("Running SHAP analysis (TreeExplainer)...")
# For XGBoost tree model we can use TreeExplainer
explainer = shap.TreeExplainer(best)
# Use a sample of train for speed
sample_for_shap = X_train_p if X_train_p.shape[0] <= 2000 else X_train_p[np.random.choice(X_train_p.shape[0], 2000, replace=False)]
shap_values = explainer.shap_values(sample_for_shap)

# summary plot
plt.figure(figsize=(10,6))
try:
    shap.summary_plot(shap_values, features=sample_for_shap, feature_names=feature_names, show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(VIS_DIR, "shap_summary.png"), dpi=150)
    plt.close()
except Exception as e:
    print("Could not create SHAP summary plot inline due to environment; saving via matplotlib fallback.")
    # fallback: shap.summary_plot writes to plt if show=False; above should work in most envs

# bar plot (mean abs shap)
plt.figure(figsize=(8,6))
try:
    shap.summary_plot(shap_values, features=sample_for_shap, feature_names=feature_names, plot_type="bar", show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(VIS_DIR, "shap_bar.png"), dpi=150)
    plt.close()
except Exception as e:
    print("Could not create SHAP bar plot:", e)

# Save shap values (small sample)
np.savez_compressed(os.path.join(MODEL_DIR, "shap_sample.npz"), shap=shap_values, features=sample_for_shap)

# Also get model's intrinsic feature importance (gain)
# mapping feature importance keys to feature names
try:
    booster = best.get_booster()
    fmap = booster.get_score(importance_type='gain')
    # booster feature names like 'f0','f1' correspond to our feature ordering
    intrinsic_importance = {}
    for k,v in fmap.items():
        # k like 'f0' -> index
        idx = int(k[1:])
        if idx < len(feature_names):
            intrinsic_importance[feature_names[idx]] = v
except Exception:
    # fallback: use feature_importances_ (approx)
    intrinsic_importance = dict(zip(feature_names, best.feature_importances_))

# Sort and keep top 20
intrinsic_sorted = sorted(intrinsic_importance.items(), key=lambda x: x[1], reverse=True)[:20]

# ---------- Local LIME explanations ----------
print("Running LIME explanations for 5 borderline cases...")
# Prepare a numpy dataset and labels for LIME (uses original scale of preprocessed features)
X_all_p = np.vstack([X_train_p, X_test_p])
y_all = np.concatenate([y_train, y_test])

# Build Lime explainer using the original training data before transform? LIME expects original feature matrix
# We'll create a wrapper that explains on preprocessed numeric vector and uses feature names above.
explainer_lime = LimeTabularExplainer(
    training_data = X_train_p,
    feature_names = feature_names,
    class_names = ['non_default','default'],
    mode = 'classification',
    discretize_continuous = False,
    random_state = RANDOM_STATE
)

# Identify borderline samples: those in test set with predicted probability close to 0.5
probs = y_pred_proba
test_indices = np.arange(len(y_test))
# sort by closeness to 0.5
dist_to_boundary = np.abs(probs - 0.5)
ordered = np.argsort(dist_to_boundary)

# Choose 5 samples: top 5 closest to 0.5
n_cases = 5
chosen_idxs = ordered[:n_cases]
# But we want 3 high-risk rejections and 2 low-risk approvals in the set if possible:
# define high-risk rejection: model predicts default (pred==1) or prob >0.5; low-risk approval: pred==0
chosen_cases = []
high_count = 0
low_count = 0
for idx in ordered:
    if len(chosen_cases) >= n_cases:
        break
    if y_pred[idx] == 1 and high_count < 3:
        chosen_cases.append(idx); high_count += 1
    elif y_pred[idx] == 0 and low_count < 2:
        chosen_cases.append(idx); low_count += 1
# If not enough, fill with nearest ones
i = 0
while len(chosen_cases) < n_cases:
    if ordered[i] not in chosen_cases:
        chosen_cases.append(ordered[i])
    i += 1

lime_results = []
for i_case, ti in enumerate(chosen_cases):
    xi = X_test_p[ti]
    yi = y_test.iloc[ti] if hasattr(y_test, 'iloc') else y_test[ti]
    prob = y_pred_proba[ti]
    pred = y_pred[ti]
    exp = explainer_lime.explain_instance(xi, best.predict_proba, num_features=10)
    # save figure
    fig = exp.as_pyplot_figure()
    fig.savefig(os.path.join(VIS_DIR, f"lime_case_{i_case+1}.png"), bbox_inches='tight', dpi=150)
    plt.close(fig)
    # capture explanation as list (feature, weight)
    exp_list = exp.as_list()
    lime_results.append({
        "case_index_in_test": int(ti),
        "id": int(ids_test.iloc[ti]) if hasattr(ids_test, 'iloc') else int(ids_test.iloc[ti]) if hasattr(ids_test, 'iloc') else int(ti),
        "true_label": int(yi),
        "predicted": int(pred),
        "probability": float(prob),
        "explanation": exp_list
    })

# ---------- Create textual report ----------
report_lines = []
report_lines.append("Credit Risk Model & Interpretability Report")
report_lines.append("="*60)
report_lines.append(f"Data file: {DATA_FILE}")
report_lines.append(f"Rows: {len(df)} | Features: {len(feature_names)}")
report_lines.append("\nMODEL PERFORMANCE (Test set)")
report_lines.append("-"*40)
report_lines.append(f"AUC: {auc:.4f}")
report_lines.append(f"F1: {f1:.4f}")
report_lines.append(f"Precision: {precision:.4f}")
report_lines.append(f"Recall: {recall:.4f}")
report_lines.append("Confusion matrix:")
report_lines.append(str(cm))
report_lines.append("\nINTRINSIC MODEL FEATURE IMPORTANCE (top 10)")
for feat, val in intrinsic_sorted[:10]:
    report_lines.append(f"{feat}: {val:.6f}")

# SHAP top features (mean abs)
try:
    mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
    shap_ranking = sorted(list(zip(feature_names, mean_abs_shap)), key=lambda x: x[1], reverse=True)[:15]
    report_lines.append("\nSHAP TOP FEATURES (global importance)")
    for feat, val in shap_ranking:
        report_lines.append(f"{feat}: {val:.6f}")
except Exception:
    report_lines.append("\nSHAP values not available for ranking.")

report_lines.append("\nLIME Local Explanations (selected cases):")
for i_case, res in enumerate(lime_results):
    report_lines.append(f"\nCase #{i_case+1} (test_index={res['case_index_in_test']}, id={res['id']})")
    report_lines.append(f"True label: {res['true_label']}, Predicted: {res['predicted']}, Prob(default): {res['probability']:.4f}")
    report_lines.append("Top local contributions (feature, weight):")
    for feat, weight in res['explanation']:
        report_lines.append(f"  {feat}: {weight:.4f}")

# Add interpretative comparison (brief)
report_lines.append("\nINTERPRETATION / POLICY IMPLICATIONS")
report_lines.append("-"*40)
report_lines.append("1) SHAP (global) identifies features that most affect predictions across the population.")
report_lines.append("2) LIME (local) reveals case-specific drivers; sometimes top global features do not appear in a given local explanation.")
report_lines.append("3) For policy: use SHAP to set broad risk rules and LIME to audit individual decisions, especially near-boundary rejections/approvals.")
report_lines.append("4) When SHAP and LIME disagree for a borderline case, flag for manual review or request additional data.")

report_text = "\n".join(report_lines)
with open(os.path.join(REPORTS_DIR, "interpretation_report.txt"), "w") as f:
    f.write(report_text)

print("\nSaved textual report to", os.path.join(REPORTS_DIR, "interpretation_report.txt"))

# Save LIME results as csv/json
lime_df_rows = []
for r in lime_results:
    # flatten explanation to string
    expl_str = "; ".join([f"{feat} ({weight:.4f})" for feat, weight in r['explanation']])
    lime_df_rows.append({
        "case_index_in_test": r['case_index_in_test'],
        "id": r['id'],
        "true_label": r['true_label'],
        "predicted": r['predicted'],
        "probability": r['probability'],
        "explanation": expl_str
    })
lime_df = pd.DataFrame(lime_df_rows)
lime_df.to_csv(os.path.join(REPORTS_DIR, "lime_local_explanations.csv"), index=False)
print("Saved LIME results to", os.path.join(REPORTS_DIR, "lime_local_explanations.csv"))

# Print short summary for user
print("\nSummary:")
print(f"- Model AUC: {auc:.4f}, F1: {f1:.4f}")
print(f"- SHAP visuals saved: {os.path.join(VIS_DIR,'shap_summary.png')}, {os.path.join(VIS_DIR,'shap_bar.png')}")
for i in range(len(lime_results)):
    print(f"- LIME case {i+1} figure: {os.path.join(VIS_DIR, f'lime_case_{i+1}.png')}")

print("\nAll done. Check the visuals and reports folder.")


Loading dataset...
Dataset shape: (500, 9)
Target distribution:
default
0    402
1     98
Name: count, dtype: int64
Numeric columns: ['loan_amnt', 'annual_income', 'dti', 'credit_score', 'employment_length', 'num_open_accounts', 'recent_missed_payments']
Categorical columns: ['purpose']
Fitting preprocessor...
Training XGBoost classifier...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best params: {'subsample': 0.6, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.6}
Best CV AUC: 0.5179216479839533

Test Metrics:
AUC: 0.5700
F1: 0.0000
Precision: 0.0000
Recall: 0.0000
Confusion matrix:
 [[80  0]
 [20  0]]

Classification report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89        80
           1       0.00      0.00      0.00        20

    accuracy                           0.80       100
   macro avg       0.40      0.50      0.44       100
weighted avg       0.64      0.80      0.